In [ ]:
import pandas as pd
import time
import sys

from datetime import datetime, timedelta
from selenium import webdriver
from bs4 import BeautifulSoup

## Scraping the HTML Page

In [ ]:
url = 'https://esriksa-emapstc.maps.arcgis.com/apps/opsdashboard/index.html#/6cd8cdcc73ab43939709e12c19b64a19'

sys.stdout.write("\r1%")
sys.stdout.flush()

driver = webdriver.Chrome('./chromedriver')

sys.stdout.write("\r5%")
sys.stdout.flush()

driver.get(url)
for i in range(0, 10):
    time.sleep(0.65) # Waiting for the page to fully load
    sys.stdout.write('\r{}%'.format(i*10 + 10))
    sys.stdout.flush()
sys.stdout.write("\n") 

soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

In [ ]:
# Date Format is "day-month" e.g "Apr 5"
update_time = datetime.strftime(datetime.now() - timedelta(1), '%b %d')
feature_list = soup.find_all('nav', class_='feature-list')

In [ ]:
all_stat = []

for feature in feature_list:
    stat = {}
    for p in feature.find_all('p'):
        strong = p.find_all('strong')
        s = strong[0].text.split(' :')
        s[0] = s[0].strip().replace(',', '')
        stat[s[1].strip('\u200e')] = int(s[0])

    all_stat.append(stat)


## Writing the new update into Dataframes

In [ ]:
total_df = pd.DataFrame.from_dict(all_stat[0], orient='index').reset_index()
total_df.columns = ['Region', 'Total Cases']
active_df = pd.DataFrame.from_dict(all_stat[1], orient='index', columns=['Active Cases']).reset_index()
active_df.columns = ['Region', 'Active Cases']
recovered_df = pd.DataFrame.from_dict(all_stat[2], orient='index', columns=['Recovered Cases']).reset_index()
recovered_df.columns = ['Region', 'Recovered Cases']

In [ ]:
# Merging the new updates into one Dataframe
new_update = pd.merge(pd.merge(total_df, active_df, how='outer'), recovered_df, how='outer')
new_update['date'] = update_time
new_update = new_update.fillna(0)

## Reading the old data files and appending the new changes

In [ ]:
daily_changes = pd.read_excel('ksa_covid19.xlsx', sheet_name='Daily Changes', parse_dates=['date'])
daily_changes = daily_changes.append(new_update, ignore_index=True)

writer = pd.ExcelWriter('ksa_covid19.xlsx')
daily_changes.to_excel(writer, 'Daily Changes', index=False)

## Merging Total Cases,	Active Cases, and Recovered Cases in Individual sheets

In [ ]:
total_df = pd.DataFrame.from_dict(all_stat[0], orient='index').reset_index()
total_df.columns = ['City', 'Total Cases {}'.format(update_time)]
active_df = pd.DataFrame.from_dict(all_stat[1], orient='index', columns=['Active Cases']).reset_index()
active_df.columns = ['City', 'Active Cases {}'.format(update_time)]
recovered_df = pd.DataFrame.from_dict(all_stat[2], orient='index', columns=['Recovered Cases']).reset_index()
recovered_df.columns = ['City', 'Recovered Cases {}'.format(update_time)]
new_update = pd.merge(pd.merge(total_df, active_df, how='outer'), recovered_df, how='outer')
new_update = new_update.fillna(0)
new_update['Death Cases {}'.format(update_time)] = new_update['Total Cases {}'.format(update_time)] - (new_update['Recovered Cases {}'.format(update_time)] + new_update['Active Cases {}'.format(update_time)])

death_df = new_update[['City', 'Death Cases {}'.format(update_time)]]

In [ ]:
total_df.columns = ['City', '{}'.format(update_time)]
active_df.columns = ['City', '{}'.format(update_time)]
recovered_df.columns = ['City', '{}'.format(update_time)]
death_df.columns = ['City', '{}'.format(update_time)]

In [ ]:
# Reading the previous update
last_all_cases = pd.read_excel('last_update.xlsx', sheet_name='Infected')
last_active_cases = pd.read_excel('last_update.xlsx', sheet_name='Active Cases')
last_recovered_cases = pd.read_excel('last_update.xlsx', sheet_name='Recovered')
last_death_cases = pd.read_excel('last_update.xlsx', sheet_name='Deaths')

In [ ]:
update_all_cases = pd.merge(last_all_cases, total_df, how='outer', on='Region')
update_active_cases = pd.merge(last_active_cases, active_df, how='outer', on='Region')
update_recovered_cases = pd.merge(last_recovered_cases, recovered_df, how='outer', on='Region')
update_death_cases = pd.merge(last_death_cases, death_df, how='outer', on='Region')

update_all_cases = update_all_cases.fillna(0)
update_active_cases = update_active_cases.fillna(0)
update_recovered_cases = update_recovered_cases.fillna(0)
update_death_cases = update_death_cases.fillna(0)

In [ ]:
update_all_cases.to_excel(writer, 'Infected', index=False)
update_active_cases.to_excel(writer, 'Active Cases', index=False)
update_recovered_cases.to_excel(writer, 'Recovered', index=False)
update_death_cases.to_excel(writer, 'Deaths', index=False)

In [ ]:
last_all_cases_changes = pd.read_excel('last_update.xlsx', sheet_name='Infected Daily')
last_active_cases_changes = pd.read_excel('last_update.xlsx', sheet_name='Active Cases Changes')
last_recovered_cases_changes = pd.read_excel('last_update.xlsx', sheet_name='Recovered Daily')
last_death_cases_changes = pd.read_excel('last_update.xlsx', sheet_name='Death Daily')

In [ ]:
update_all_cases['{}'.format(update_time)] = update_all_cases[update_all_cases.columns[-1]] - update_all_cases[update_all_cases.columns[-2]]
update_active_cases['{}'.format(update_time)] = update_active_cases[update_active_cases.columns[-1]] - update_active_cases[update_active_cases.columns[-2]]
update_recovered_cases['{}'.format(update_time)] = update_recovered_cases[update_recovered_cases.columns[-1]] - update_recovered_cases[update_recovered_cases.columns[-2]]
update_death_cases['{}'.format(update_time)] = update_death_cases[update_death_cases.columns[-1]] - update_death_cases[update_death_cases.columns[-2]]


In [ ]:
update_all_cases_changes = pd.merge(last_all_cases_changes, update_all_cases[['{}'.format(update_time), 'Region']], how='outer', on='Region')
update_active_cases_changes = pd.merge(last_active_cases_changes, update_active_cases[['{}'.format(update_time), 'Region']], how='outer', on='Region')
update_recovered_cases_changes = pd.merge(last_recovered_cases_changes, update_recovered_cases[['{}'.format(update_time), 'Region']], how='outer', on='Region')
update_death_cases_changes = pd.merge(last_death_cases_changes, update_death_cases[['{}'.format(update_time), 'Region']], how='outer', on='Region')


In [ ]:
update_all_cases_changes.to_excel(writer, 'Infected Daily', index=False)
update_active_cases_changes.to_excel(writer, 'Active Cases Changes', index=False)
update_recovered_cases_changes.to_excel(writer, 'Recovered Daily', index=False)
update_death_cases_changes.to_excel(writer, 'Death Daily', index=False)


In [ ]:
writer.save()
print('Finished Scraping, check the results in last_update.xlsx')